In [1]:
import numpy as np
import pandas as pd
import os
os.environ["KERAS_BACKEND"] = "pytorch"
import keras
from keras import layers
!pip install rdkit-pypi==2023.3.1b1 --upgrade
from rdkit import Chem

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 10.8 MB/s eta 0:00:00


In [2]:
!mkdir ~/.kaggle
!echo '{"username":"michaelmoser1","key":"fb607e421aa0b4036f30d872f4f26f92"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle

In [3]:
! kaggle competitions download leash-BELKA

100% 4.15G/4.16G [00:42<00:00, 145MB/s]
100% 4.16G/4.16G [00:43<00:00, 104MB/s]


In [4]:
! unzip leash-BELKA.zip

Archive:  leash-BELKA.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: test.parquet            
  inflating: train.csv               
  inflating: train.parquet           


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

model = Sequential([
    Dense(128, activation='relu', input_dim=K),
    Dense(64, activation='relu'),
    Dense(3, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

NameError: name 'process_chunk' is not defined

In [16]:
def X_processing(pca, X):
  pca.fit(X)
  return pca.transform(X)

In [27]:
from rdkit.Chem import Descriptors
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# chunk_size = 100
# df = pd.read_csv('train.csv', chunksize=chunk_size)
# i = 0
# for chunk in df:
#   descriptors = [Descriptors.CalcMolDescriptors(Chem.MolFromSmiles(mol)) for mol in chunk['molecule_smiles'].tolist()]
#   features_df = pd.DataFrame(descriptors)
#   X = features_df.to_numpy()
#   Y =
#   pca = PCA(n_components=209)
#   pca.fit(X)
#   break

import pandas as pd

chunk_size = 1000
df = pd.read_csv('train.csv', chunksize=chunk_size)

descriptor_names = [desc[0] for desc in Descriptors.descList]

# Assuming df is defined earlier as a pandas DataFrame iterator
# e.g., df = pd.read_csv('train.csv', chunksize=chunk_size)
for chunk in df:
  pivot = chunk.pivot(index='molecule_smiles', columns='protein_name', values='binds')
  pivot.reset_index(inplace=True)
  pivot['Descriptors'] = pivot['molecule_smiles'].apply(lambda mol: [desc[1](Chem.MolFromSmiles(mol)) for desc in Descriptors.descList])

  descriptors_df = pd.DataFrame(pivot['Descriptors'].tolist(), columns=descriptor_names, index=pivot.index)
  final_df = pd.concat([pivot, descriptors_df], axis=1)
  X = final_df.drop(columns=['molecule_smiles'] + list(pivot.columns.get_level_values(0))).to_numpy()
  Y = pivot[['BRD4', 'HSA', 'sEH']].to_numpy()

  X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
  Y = np.nan_to_num(Y, nan=0.0, posinf=0.0, neginf=0.0)

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  pca = PCA(n_components=209)
  pca.fit(X)
  cum_var_ratio = pca.explained_variance_ratio_.cumsum()
  K = (cum_var_ratio >= 0.85).argmax() + 1
  print("Number of features for 85% energy:", K)

  pca = PCA(n_components=K)
  Theta_train = X_processing(pca, X)

  break


Number of features for 85% energy: 33
